# 工具调用

In [1]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass

os.environ["LANGSMITH_TRACING"] = "true"
if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass(
        prompt="Enter your LangSmith API key (optional): "
    )
if "LANGSMITH_PROJECT" not in os.environ:
    os.environ["LANGSMITH_PROJECT"] = getpass.getpass(
        prompt='Enter your LangSmith Project Name (default = "default"): '
    )
    if not os.environ.get("LANGSMITH_PROJECT"):
        os.environ["LANGSMITH_PROJECT"] = "default"
if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass(
        prompt="Enter your OpenAI API key (required if using OpenAI): "
    )

base_url = os.environ.get("BASE_URL")
model_name = os.environ.get("MODEL_NAME")
deepseek_model_name = os.environ.get("DEEPSEEK_MODEL_NAME")

print(base_url, model_name)

https://dashscope.aliyuncs.com/compatible-mode/v1 qwen-plus


In [2]:


from langchain_deepseek import ChatDeepSeek



llm = ChatDeepSeek(
    model=deepseek_model_name,
    temperature=0,
    max_retries=2,
    base_url=base_url,
)

In [10]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

## 添加工具
或者使用 pydantic
```python
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")
```
或者使用 TypedDict class
```python
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]
```

In [12]:
tools = [add, multiply]
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_a34460bd-6f77-47c8-9381-b84d5b759515', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 23, 'prompt_tokens': 246, 'total_tokens': 269, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 192}, 'prompt_cache_hit_tokens': 192, 'prompt_cache_miss_tokens': 54}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': 'fa40d985-0408-4ca2-8950-066efc23a733', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--972fb62d-bc41-4dd6-b479-3fa1a29d982e-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_0_a34460bd-6f77-47c8-9381-b84d5b759515', 'type': 'tool_call'}], usage_metadata={'input_tokens': 246, 'output_tokens': 23, 'total_tokens': 269, 'input_token_details': {'cache_rea

### Tool calls
如果大模型的返回中调用了工具，则可以使用 .tool_calls 从返回的 message 后者 message chunk 中获取工具调用的列表

In [13]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_0_410a913b-aef5-4d6c-850e-b51703054416',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_1_5ff473dc-8010-4134-92d5-ca5b183a4f9f',
  'type': 'tool_call'}]

#### 工具的格式化输出
使用PydanticToolsParser可以将 tool_calls 的返回

In [14]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]

## 简短 prompt tool

In [4]:
from langchain_core.tools import tool


@tool
def add(a: int, b: int) -> int:
    """Adds a and b."""
    return a + b


@tool
def multiply(a: int, b: int) -> int:
    """Multiplies a and b."""
    return a * b


tools = [add, multiply]

In [5]:
llm_with_tools = llm.bind_tools(tools)


In [6]:
llm_with_tools.invoke(
    "Whats 119 times 8 minus 20. Don't do any math yourself, only use tools for math. Respect order of operations"
).tool_calls

[{'name': 'multiply',
  'args': {'a': 119, 'b': 8},
  'id': 'call_0_88a5ef48-e575-48e5-bc3e-10c3b494cd82',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 20, 'b': 0},
  'id': 'call_1_e23524ee-4547-4cfe-9db8-db7ef63e2828',
  'type': 'tool_call'}]

## How to bind model-specific tools

In [7]:
model_with_tools = llm.bind(
    tools=[
        {
            "type": "function",
            "function": {
                "name": "multiply",
                "description": "Multiply two integers together.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "a": {"type": "number", "description": "First integer"},
                        "b": {"type": "number", "description": "Second integer"},
                    },
                    "required": ["a", "b"],
                },
            },
        }
    ]
)


In [9]:
model_with_tools.invoke("What is 3 * 12?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0_9be24576-db8f-4954-96eb-7f0b05bc0fc9', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function', 'index': 0}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 140, 'total_tokens': 162, 'completion_tokens_details': None, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 128}, 'prompt_cache_hit_tokens': 128, 'prompt_cache_miss_tokens': 12}, 'model_name': 'deepseek-chat', 'system_fingerprint': 'fp_8802369eaa_prod0425fp8', 'id': 'f2f83bcc-4f2f-422a-90ef-de86265572ec', 'service_tier': None, 'finish_reason': 'tool_calls', 'logprobs': None}, id='run--4f694e6b-4ee7-42ea-850b-f74b9cc2639c-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_0_9be24576-db8f-4954-96eb-7f0b05bc0fc9', 'type': 'tool_call'}], usage_metadata={'input_tokens': 140, 'output_tokens': 22, 'total_tokens': 162, 'input_token_details': {'cache_rea